In [1]:
import optuna
import os
from pathlib import Path
import kaggle
import pandas as pd
import numpy as np
from sklearn.preprocessing import FunctionTransformer, PowerTransformer, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import HistGradientBoostingClassifier


pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
if iskaggle:
    path = Path('../input/playground-series-s3e23')
    orig_path = Path('../input/software-defect-prediction')
else:
    path = Path('../input/playground-series-s3e23')
    orig_path = Path('../input/software-defect-prediction')
    if not path.exists():
        kaggle.api.competition_download_cli(str(path))
        zipfile.ZipFile(f'{path}.zip').extractall(path)
    if not orig_path.exists():
        kaggle.api.competition_download_cli(str(path))
        zipfile.ZipFile(f'{path}.zip').extractall(path)

SEED = 42
SPLITS = 5
VERSION = 1
OUTPUT = f'hist_grad_boost_clf_submisson_v{VERSION}.csv'
N_TRIALS = 30

np.random.seed(SEED)

/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train = pd.read_csv(path/'train.csv', index_col = 'id')
test = pd.read_csv(path/'test.csv', index_col = 'id')
orig_train = pd.read_csv(orig_path/'jm1.csv')

In [3]:
for object_features in list(orig_train.loc[:, orig_train.dtypes == 'O']):
    orig_train[object_features] = orig_train[object_features].replace({'?' : np.nan}).astype('float64')

In [4]:
X = pd.concat([train, orig_train])
y = X.pop('defects')

In [5]:
partial_pipeline = make_pipeline(SimpleImputer(), FunctionTransformer(func=np.log1p, validate=False), StandardScaler())

In [6]:
def objective(trial):
    params = {
        'random_state': trial.suggest_categorical('random_state', [SEED]),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'l2_regularization': trial.suggest_int('l2_regularization', 0, 100),
        'max_iter': trial.suggest_int('max_iter', 10, 10_000),
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 2, 500),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 2, 300),
        'max_bins': trial.suggest_int('max_bins', 32, 255),
        'class_weight': trial.suggest_categorical('class_weight', [None, 'balanced'])
    }
    pipeline = make_pipeline(partial_pipeline, HistGradientBoostingClassifier(**params))
    scores = cross_val_score(pipeline, X, y, cv=SPLITS, scoring='roc_auc', n_jobs=-1)
    return scores.mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=N_TRIALS, show_progress_bar=True)

[I 2023-10-16 13:53:39,513] A new study created in memory with name: no-name-49a93b39-ed13-481d-aeea-7db2db0da55a
Best trial: 0. Best value: 0.78462:   3%|█████▌                                                                                                                                                               | 1/30 [00:13<06:37, 13.69s/it]

[I 2023-10-16 13:53:53,208] Trial 0 finished with value: 0.7846201158377332 and parameters: {'random_state': 42, 'learning_rate': 0.024118501272207507, 'l2_regularization': 94, 'max_iter': 5889, 'max_leaf_nodes': 268, 'min_samples_leaf': 60, 'max_bins': 102, 'class_weight': None}. Best is trial 0 with value: 0.7846201158377332.


Best trial: 1. Best value: 0.785043:   7%|██████████▉                                                                                                                                                         | 2/30 [00:40<09:57, 21.34s/it]

[I 2023-10-16 13:54:19,907] Trial 1 finished with value: 0.7850426672143341 and parameters: {'random_state': 42, 'learning_rate': 0.010039788539813715, 'l2_regularization': 98, 'max_iter': 5033, 'max_leaf_nodes': 280, 'min_samples_leaf': 211, 'max_bins': 229, 'class_weight': None}. Best is trial 1 with value: 0.7850426672143341.


Best trial: 1. Best value: 0.785043:  10%|████████████████▍                                                                                                                                                   | 3/30 [00:59<09:10, 20.39s/it]

...

[I 2023-10-16 14:00:47,337] Trial 27 finished with value: 0.7856418820411478 and parameters: {'random_state': 42, 'learning_rate': 0.015220913868352361, 'l2_regularization': 74, 'max_iter': 3761, 'max_leaf_nodes': 43, 'min_samples_leaf': 203, 'max_bins': 136, 'class_weight': 'balanced'}. Best is trial 11 with value: 0.7860260421824963.


Best trial: 11. Best value: 0.786026:  97%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌     | 29/30 [07:11<00:11, 11.94s/it]

[I 2023-10-16 14:00:50,607] Trial 28 finished with value: 0.7835347804760723 and parameters: {'random_state': 42, 'learning_rate': 0.0997258224203673, 'l2_regularization': 33, 'max_iter': 2907, 'max_leaf_nodes': 139, 'min_samples_leaf': 275, 'max_bins': 150, 'class_weight': 'balanced'}. Best is trial 11 with value: 0.7860260421824963.


Best trial: 11. Best value: 0.786026: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [07:22<00:00, 14.75s/it]

[I 2023-10-16 14:01:02,030] Trial 29 finished with value: 0.7845509830320572 and parameters: {'random_state': 42, 'learning_rate': 0.02211221496420053, 'l2_regularization': 64, 'max_iter': 8524, 'max_leaf_nodes': 226, 'min_samples_leaf': 270, 'max_bins': 125, 'class_weight': 'balanced'}. Best is trial 11 with value: 0.7860260421824963.


In [7]:
study.best_params, study.best_value

({'random_state': 42,
  'learning_rate': 0.010212498830869023,
  'l2_regularization': 53,
  'max_iter': 5167,
  'max_leaf_nodes': 24,
  'min_samples_leaf': 300,
  'max_bins': 186,
  'class_weight': 'balanced'},
 0.7860260421824963)

In [8]:
pipeline = make_pipeline(partial_pipeline, HistGradientBoostingClassifier(**study.best_params))
pipeline.fit(X, y)

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('simpleimputer', SimpleImputer()),
                                 ('functiontransformer',
                                  FunctionTransformer(func=<ufunc 'log1p'>)),
                                 ('standardscaler', StandardScaler())])),
                ('histgradientboostingclassifier',
                 HistGradientBoostingClassifier(class_weight='balanced',
                                                l2_regularization=53,
                                                learning_rate=0.010212498830869023,
                                                max_bins=186, max_iter=5167,
                                                max_leaf_nodes=24,
                                                min_samples_leaf=300,
                                                random_state=42))])

In [9]:
submission = test.copy()
submission['defects'] = pipeline.predict_proba(submission)[:, 1]
submission.defects.to_csv(OUTPUT)